## Interactive Interfaces with Gradio

*[Coding along with the Udemy online course [LLM Engineering: Master AI & Large Language Models](https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models/) by Ed Donner; GitHub repo can be found at [github.com/ed-donner/llm_engineering](https://github.com/ed-donner/llm_engineering)]*

This notebook is about building user interfaces using the [Gradio framework](https://www.gradio.app/).

In [1]:
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display, update_display
import pandas as pd
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from typing import List
# and finally, let's import gradio as a new feature here
# https://www.gradio.app/guides/quickstart
import gradio as gr

In [2]:
openai_api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

anthropic_api_key = pd.read_csv("~/tmp/anthropic/anthropic-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

google_api_key = pd.read_csv("~/tmp/google-gemini/gemini-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

# connect to openai
openai = OpenAI(api_key=openai_api_key)

# connect to anthropic
claude = anthropic.Anthropic(api_key=anthropic_api_key)

# connect to gemini
google.generativeai.configure(api_key=google_api_key)

Don't be a fool and sent your api key to github
Don't be a fool and sent your api key to github
Don't be a fool and sent your api key to github


### Simple example on how to use Gradio

In [3]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [4]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [5]:
# launching a Gradip interface
# handing in the function the user interface is build aroung: `fn=shout`
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [6]:
# same with css applied
gr.Interface(
    fn=shout, 
    inputs="textbox", 
    outputs="textbox", 
    css=".gradio-container, .block {background-color: black};"
).launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [7]:
# testing some more parameters
# allow_flagging="never" which disallows flagging (what doesn't seem to happen by default)
# share=True on lauch creates a publicly url (app in Gradio server but function still running locally... it's MAGIC)
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", css=".gradio-container, .block {background-color: black};", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://7491057d8255f3152a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Shout has been called with input what has happened in the 11th century?


In [8]:
view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    css=".gradio-container, .block {background-color: black};",
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


### LLM integration in Gradio

In [9]:
# a generic system message
system_message = "You are a helpful assistant"

In [10]:
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [11]:
message_gpt("What is today's date?") # llm doesn't know :)

"Today's date is October 4, 2023."

In [12]:
# and now ... once again with calling gpt
view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    css=".gradio-container, .block {background-color: black};",
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [13]:
# once again with overwriting the system message
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [14]:
# function that creates a call for streaming back results
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [15]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


### Gradio with Claude

In [16]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [18]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [19]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [21]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
